In [ ]:

from collections import deque
from tqdm.notebook import tqdm
import re
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os


In [ ]:
train_df = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/train.csv')
test_df = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/test.csv')
print(test_df.shape)

In [ ]:
train_df[train_df.essay_id == '00066EA9880D']

In [ ]:
def basic_cleaning(text):
    """
    clear url/ not alpha/ fuck-bitch swear
    """
    text = re.sub(r'https?://www\.\S+\.cm', '', text)
    text = re.sub(r'[^a-zA-Z|\s]', '', text)
    text = re.sub(r'\*+', 'swear', text)
    return text

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    #emoticons
    #symbols & pictographs
    #transport & map symbols
    #flags (iOS)
    emoji_pattern = re.compile("["\
        u"\U0001F600-\U0001F64F|"\
        u"\U0001F300-\U0001F5FF|"\
        u"\U0001F680-\U0001F6FF|"\
        u"\U0001F1E0-\U0001F1FF|"\
        u"\U00002702-\U000027B0|"\
        u"\U000024C2-\U0001F251"\
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# remove repeated characters
def remove_multiplechars(text):
    """
    for example, so we have “way” instead of “waaaayyyyy”
    """
    text = re.sub(r'(.)\1{3,}', r'\1', text)
    return text

def clean(df):
    for col in ['discourse_text']:#,'selected_text']:
        df[col] = df[col].astype(str).apply(lambda x:basic_cleaning(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_emoji(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_html(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_multiplechars(x))
    return df.sample(frac=1)

train_df = clean(train_df)
train_df_selection = train_df.sample(frac=1)
X_tr = train_df_selection.discourse_text.values

test_df = clean(test_df)
test_df_selection = test_df.sample(frac=1)
X_te = test_df_selection.discourse_text.values
print(X_te.shape)
print('clean Done')

In [ ]:
y_tr1, uniques_tr1 = pd.factorize(train_df_selection.discourse_type, sort=True)
y_tr2, uniques_tr2 = pd.factorize(train_df_selection.discourse_effectiveness, sort=True)
print(uniques_tr1, uniques_tr2)

In [ ]:
files_list = [
    f'../input/feedback-prize-effectiveness/train/{f}' for f in os.listdir('../input/feedback-prize-effectiveness/train')
] + [
    f'../input/feedback-prize-effectiveness/test/{f}' for f in os.listdir('../input/feedback-prize-effectiveness/test')
]

In [ ]:
import gc
# del word2id, id2word, wordid_freq
gc.collect()

In [ ]:
data_g.stop_words

In [ ]:
# len(data_g.wordid_freq), data_g.wordid_freq[data_g.word2id['unk']], len(data_g.stop_words), data_g.stop_words
# sorted(data_g.wordid_freq.items(), key=lambda c:c[1], reverse=True)
# # data_g.wordid_freq

In [ ]:
# help(gensim.models.callbacks.CallbackAny2Vec).

In [ ]:
# skp_model = word2Vec(
#     sentences=None, corpus_file=None,
#     vector_size=256, alpha=0.025, min_alpha=5e-5,
#     window=5,  min_count=5, max_vocab_size=None,
#     # drop hight freq word , random drop freq_rate > 1e-5
#     sample=1e-5, seed=2022,
#     # NSG
#     negative=5, ns_exponent=0.75,
#     sg=1, hs=0, workers=3,
#     min_alpha=0.0001,
#     epochs=5, sorted_vocab=1, batch_words=4096, compute_loss=True
# )

In [ ]:
def get_batch_pair(wordid_list, batch_size, window_size, idx, word_pair_queue):
    while len(word_pair_queue) < batch_size:
        if idx ==len(wordid_list):
            idx=0
        for _ in range(len(wordid_list)):
            for i in range(max(idx - window_size, 0), min(idx + window_size+1, len(wordid_list)-1)):
                w = wordid_list[idx]
                v = wordid_list[i]
                if idx == i:
                    continue
                word_pair_queue.append((w, v))
            idx += 1
        res_pair = []
        for _ in range(batch_size):
            res_pair.append(
                word_pair_queue.popleft()
            )
        return res_pair

def get_new_batch_pair(wordid_list, window_size):
    word_pair_queue = []
    for idx in range(len(wordid_list)):
        for i in range(max(idx - window_size, 0), min(idx + window_size+1, len(wordid_list)-1)):
            w = wordid_list[idx]
            v = wordid_list[i]
            if idx == i:
                continue
            if w == v:
                continue
            word_pair_queue.append((w, v))
    return word_pair_queue


wordid_list = data_g.file2id(files_list[20])
# pos_pair = get_batch_pair(wordid_list, batch_size=32, window_size=3, idx=0, word_pair_queue=deque())
pos_pair = get_new_batch_pair(wordid_list, window_size=5)
neg_ = data_g.get_neg_samples(pos_pair, 5)
print('get_neg_samples Done')

In [ ]:
# pos_pair = []
# neg_ = []
# for f in tqdm(files_list):
#     wordid_list = data_g.file2id(f)
#     tmp_pair = get_new_batch_pair(wordid_list, window_size=5)
#     print(len(tmp_pair))
#     tmp_pair = data_g.subsample(tmp_pair)
#     print(len(tmp_pair))
#     pos_pair.extend(tmp_pair)
#     neg_.extend(data_g.get_neg_samples(tmp_pair,  5))
#     break

In [ ]:
print(len(data_g.wordid_freq))
skp_model = skipGramModel(vocab_size=len(data_g.wordid_freq), embed_size=256)

In [ ]:
pos_pair = []
neg_ = []
for f in tqdm(files_list):
    wordid_list = data_g.file2id(f)
    tmp_pair = get_new_batch_pair(wordid_list, window_size=5)
    tmp_pair = data_g.subsample(tmp_pair)
    pos_pair.extend(tmp_pair)
    neg_.extend(data_g.get_neg_samples(tmp_pair,  5))

print('get_new_batch_pair Done')
pos_pair = np.array(pos_pair)

lr_ = 0.015 # 1e-3
print('--'*25, '\nlr:', lr_ )
skp_model.compile(
    loss=nn.CrossEntropyLoss(),
    optimizer=SparseAdam,
    learning_rate=lr_,
    metrics=[accuracy_score],
    verbose=100
)
skp_model.fit(pos_pair[:, 0], pos_pair[:, 1], np.array(neg_), batch_size=2048, epochs=3)

# pos_pair = []
# neg_ = []
# for f in tqdm(files_list):
#     wordid_list = data_g.file2id(f)
#     tmp_pair = get_new_batch_pair(wordid_list, window_size=5)
#     tmp_pair = data_g.subsample(tmp_pair)
#     pos_pair.extend(tmp_pair)
#     neg_.extend(data_g.get_neg_samples(tmp_pair,  5))

# print('get_new_batch_pair Done')
# pos_pair = np.array(pos_pair)
# lr_ = 1e-3
# print('--'*25, '\nlr:', lr_ )
# skp_model.optim = SparseAdam(skp_model.parameters(), lr=lr_)
# skp_model.fit(pos_pair[:, 0], pos_pair[:, 1], np.array(neg_), batch_size=2048, epochs=1)

# lr_ = 1e-5
# print('--'*25, '\nlr:', lr_ )
# skp_model.optim = SparseAdam(skp_model.parameters(), lr=lr_)
# skp_model.fit(pos_pair[:, 0], pos_pair[:, 1], np.array(neg_), batch_size=2048, epochs=1)
# print('skp_model Done')

In [ ]:
del pos_pair, neg_
gc.collect()

In [ ]:
def my_encode(texts, maxlen=128):
    all_ids = []
    for i in tqdm(range(len(texts))):
        tmp_ids = data_g.context2id(texts[i])
        tmp_ids = tmp_ids[:maxlen] + [0] * (maxlen - len(tmp_ids))
        all_ids.append(tmp_ids)
    return np.array(all_ids)

X = my_encode(
    X_tr,
    maxlen=128
)

X_te_enc = my_encode(
    X_te,
    maxlen=128
)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.optim import AdamW, Adam, RMSprop
from sklearn.metrics import accuracy_score
import typing  as typ

embed_1 = skp_model.w_embed.weight.data.cpu().numpy()
embed_2 = skp_model.v_embed.weight.data.cpu().numpy()
embed = (embed_1 + embed_2) / 2


class compDataset(Dataset):
    def __init__(self, encodings, label_i, label_j):
        self.encodings = encodings
        self.label_i = label_i
        self.label_j = label_j


    def __getitem__(self, idx):
        ids =  torch.tensor(self.encodings[idx]).long()
        s_idx = torch.Tensor(range(len(ids)))
        s_mask = torch.ones((len(ids), 256)).float()
        if sum(ids != 0):
            st = int(s_idx[ids != 0].max(dim=0).values + 1)
            if st < len(ids):
                s_mask[st:] = 1e-9
        item = {'input_ids': ids}
        item['mask'] = s_mask
        item['label_i'] = torch.tensor(int(self.label_i[idx]))
        item['label_j'] = torch.tensor(int(self.label_j[idx]))
        return item

    def __len__(self):
        return len(self.label_i)



class multiTaskModel(nn.Module):
    def __init__(self, num_labels_i, num_labels_j):
        super(multiTaskModel, self).__init__()
        self.embed = nn.Embedding.from_pretrained(
            torch.Tensor(embed).float(), freeze=True
        )
        self.lstm_layer = nn.LSTM(input_size=256, hidden_size=128, bidirectional=True)
        self.lstm_layer2 = nn.LSTM(input_size=128*2, hidden_size=64, bidirectional=True)
        self.drop1 = nn.Dropout(0.3)
        self.fc1 = nn.Linear(64*2, 32)
        self.relu = nn.ReLU(inplace=True)
        self.drop2 = nn.Dropout(0.3)
        self.sgm1 = nn.Sigmoid()
        self.sgm2 = nn.Sigmoid()
        self.classifier_i = nn.Linear(32, num_labels_i)
        self.classifier_j = nn.Linear(32, num_labels_j)
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        self.device = torch.device("cpu")
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name:
                if 'weight' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)

    def forward(self, inputs, mask):
        out = self.embed(inputs)
        out = torch.mul(out, mask)
        out,(h,c) = self.lstm_layer(out)
        out,(h,c) = self.lstm_layer2(out)
        out = out.max(axis=1).values
        out=self.drop1(out)
        out=self.fc1(out)
        out=self.relu(out)
        out=self.drop2(out)
#         logits_i = self.sgm1(torch.clamp(self.classifier_i(out), min=-10, max=10))
#         logits_j = self.sgm2(torch.clamp(self.classifier_j(out), min=-10, max=10))
        logits_i = self.classifier_i(out)
        logits_j = self.classifier_j(out)
        return logits_i, logits_j

    def compile(self,
                loss: typ.Callable,
                optimizer: typ.Callable,
                learning_rate: float,
                metrics: typ.List[typ.Callable],
                verbose: int=100):
        self.verbose = verbose
        b=torch.cuda.is_available()
        if(b):
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")

        self.to(self.device)
        self.loss_fn = loss
        self.optim = optimizer(self.parameters(), lr=learning_rate)
        self.metrics = metrics
        self.iter_num = 0

    def _training_step(self, x, y_i, y_j, mask):
        x = x.to(self.device)
        y_i = y_i.to(self.device)
        y_j = y_j.to(self.device)
        mask = mask.to(self.device)
        self.epoch_steps += 1
        self.epoch_samples += len(y_i)
        # 正向传播
        self.optim.zero_grad()
        pred_i, pred_j =  self.forward(x, mask)
        loss_i = self.loss_fn(pred_i, y_i)
        loss_j = self.loss_fn(pred_j, y_j)
        loss = 0.2 * loss_i + 0.8 * loss_j

        pred_i = pred_i.cpu().detach().numpy()
        pred_j = pred_j.cpu().detach().numpy()
        self.epoch_loss += loss.detach().cpu().numpy()
        self.epoch_focus_loss += loss_j.detach().cpu().numpy()
        self.epoch_right_samples += np.sum(np.argmax(pred_i, axis=1) == y_i.cpu().detach().numpy() )
        self.epoch_right_samples_j += np.sum(np.argmax(pred_j, axis=1) == y_j.cpu().detach().numpy() )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.parameters(), 1.0)

        self.optim.step()
        self.iter_num += 1
        if(self.iter_num % self.verbose == 0):
            metric_str = ''
            for metric_fn in self.metrics:
                metric_res_i = metric_fn(np.argmax(pred_i, axis=1), y_i.cpu().detach().numpy())
                metric_res_j = metric_fn(np.argmax(pred_j, axis=1), y_j.cpu().detach().numpy())
                metric_str += f'{metric_fn.__name__}-i : {metric_res_i:.3f}, {metric_fn.__name__}-j : {metric_res_j:.3f},'

            print(f"[ iter_num-{self.iter_num} ]: loss: {loss:.5f} loss_j: {loss_j:.5f} " + metric_str)
        return self.epoch_loss, self.epoch_focus_loss

    def train_one_epoch(self, dataloader, ep_idx=0):
        self.epoch_loss = 0
        self.epoch_samples = 0
        self.epoch_right_samples = 0
        self.epoch_right_samples_j = 0
        self.epoch_steps = 0
        self.epoch_focus_loss = 0

        tqdm_bar = tqdm(dataloader)
        for batch in tqdm_bar:
            loss_tt, loss = self._training_step(batch['input_ids'], batch['label_i'], batch['label_j'], batch['mask'])
            loss_p = loss / self.epoch_steps
            loss_tt = loss_tt / self.epoch_steps
            tqdm_bar.set_description(f'[ Train-Epoch {ep_idx} ]')
            tqdm_bar.set_postfix(loss=f'total:{loss_tt:.5f} focus_loss:{loss_p:.5f}')
            tqdm_bar.update()


        self.epoch_loss /= self.epoch_steps
        self.epoch_focus_loss /= self.epoch_steps
        acc_i = self.epoch_right_samples / self.epoch_samples
        acc_j = self.epoch_right_samples_j / self.epoch_samples
        print(f"[ epoch: {ep_idx} ] loss_tt: {self.epoch_loss:.5f}, loss_j: {self.epoch_focus_loss:.5f}, acci: {acc_i:.3f}, accj: {acc_j:.3f}")

    def _valitate_step(self, x, y_i, y_j, mask):
        with torch.no_grad():
            x = x.to(self.device)
            y_i = y_i.to(self.device)
            y_j = y_j.to(self.device)
            mask = mask.to(self.device)
            self.epoch_steps += 1
            self.epoch_samples += len(y_i)
            pred_i, pred_j =  self.forward(x, mask)
            loss = self.loss_fn(pred_i, y_i) + self.loss_fn(pred_j, y_j)

            pred_i = pred_i.cpu().detach().numpy()
            pred_j = pred_j.cpu().detach().numpy()
            self.epoch_loss += loss.detach().cpu().numpy()
            self.epoch_right_samples += np.sum(np.argmax(pred_i, axis=1) == y_i.cpu().detach().numpy() )
            self.epoch_right_samples_j += np.sum(np.argmax(pred_j, axis=1) == y_j.cpu().detach().numpy() )
            self.iter_num += 1
            if(self.iter_num % self.verbose == 0):
                metric_str = ''
                for metric_fn in self.metrics:
                    metric_res_i = metric_fn(np.argmax(pred_i, axis=1), y_i.cpu().detach().numpy())
                    metric_res_j = metric_fn(np.argmax(pred_j, axis=1), y_j.cpu().detach().numpy())
                    metric_str += f'{metric_fn.__name__}-i : {metric_res_i:.3f}, {metric_fn.__name__}-j : {metric_res_j:.3f},'

                print(f"[ iter_num-{self.iter_num} ]: loss: {loss:.5f}  " + metric_str)

    def valitate_one_epoch(self, dataloader, ep_idx=0):
        self.epoch_loss = 0
        self.epoch_samples = 0
        self.epoch_right_samples = 0
        self.epoch_right_samples_j = 0
        self.epoch_steps = 0
        for batch in tqdm(dataloader):
            self._valitate_step(batch['input_ids'], batch['label_i'], batch['label_j'], batch['mask'])

        self.epoch_loss /= self.epoch_steps
        acc_i = self.epoch_right_samples / self.epoch_samples
        acc_j = self.epoch_right_samples_j / self.epoch_samples
        print("-------------------------------")
        print(f"[ epoch: {ep_idx}-valitation ] loss_tt: {self.epoch_loss:.5f}, acci: {acc_i:.3f}, accj: {acc_j:.3f}")
        print("-------------------------------")

    def fit(self, X, yi, yj,
            batch_size: int=32,
            epochs: int=5,
            validation_split: float=0.1
            ):

        sp = StratifiedShuffleSplit(n_splits=epochs, test_size=int(len(yi)*validation_split))
        for idx, (tr_idx, te_idx) in  enumerate(sp.split(X, yi)):
            tr_data = compDataset(X[tr_idx, :], yi[tr_idx], yj[tr_idx])
            val_data = compDataset(X[te_idx, :], yi[te_idx], yj[te_idx])
            train_loader = DataLoader(tr_data, batch_size=batch_size, shuffle=True)
            val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
            self.train_one_epoch(train_loader, idx+1)
            self.valitate_one_epoch(val_dataloader, idx+1)


In [ ]:
model = multiTaskModel(len(uniques_tr1), len(uniques_tr2))
model.compile(
    loss=nn.CrossEntropyLoss(),
    optimizer=AdamW, # RMSprop, #
    learning_rate=1e-3 , #5e-4
    metrics=[accuracy_score],
    verbose=100
)

In [ ]:
batch_size = 128
model.fit(X, y_tr1, y_tr2,
        batch_size=batch_size,
        epochs=20,
        validation_split=0.2
)

# lr_ = 5e-5
# print('--'*25, '\nlr:', lr_ )
# model.optim = AdamW(model.parameters(), lr=lr_) #RMSprop(model.parameters(), lr=lr_)#
# model.fit(X, y_tr1, y_tr2,
#         batch_size=batch_size,
#         epochs=10,
#         validation_split=0.2
# )

In [ ]:
def predict(teloaders):
    pred_i_list = []
    pred_j_list = []
    for batch in teloaders:
        x, y_i, y_j, mask = batch['input_ids'], batch['label_i'], batch['label_j'], batch['mask']
        with torch.no_grad():
            x = x.to(model.device)
            y_i = y_i.to(model.device)
            y_j = y_j.to(model.device)
            mask = mask.to(model.device)
            pred_i, pred_j =  model.forward(x, mask)
            pred_i = pred_i.cpu().detach().numpy()
            pred_j = pred_j.cpu().detach().numpy()
            pred_i_list.append(pred_i)
            pred_j_list.append(pred_j)
    return np.concatenate(pred_i_list), np.concatenate(pred_j_list)



final_test_data = compDataset(X_te_enc, [0]*len(X_te_enc), [0]*len(X_te_enc))
teloaders = DataLoader(final_test_data, batch_size=batch_size, shuffle=False)
pred_i_out, pred_j_out = predict(teloaders)

In [ ]:
# with torch.no_grad():
#     aa = pred_j_out + np.abs(pred_j_out.min(axis=1, keepdims=True))*1.25
#     aa = aa[:, [2, 0, 1]]
#     my_norm_pred = np.clip(np.exp(aa) / np.exp(aa).sum(axis=1, keepdims=True), a_min=1e-15, a_max=1-1e-15)
#     norm_pred = np.array([
#             [0.0142, 0.3949, 0.5909],
#             [0.0380, 0.8436, 0.1184],
#             [0.0301, 0.7221, 0.2478],
#             [0.0692, 0.6866, 0.2441],
#             [0.0585, 0.6580, 0.2835],
#             [0.0217, 0.4150, 0.5633],
#             [0.0200, 0.3050, 0.6751],
#             [0.0309, 0.6665, 0.3026],
#             [0.0299, 0.4357, 0.5344],
#             [0.0237, 0.5962, 0.3801]
#             ])
#     print('lab 0.61 loss:',model.loss_fn(
#         torch.Tensor(norm_pred).float(),
#         torch.LongTensor([2, 1, 1, 1, 1, 2, 2, 1, 2, 1])))
#     print('My loss:',model.loss_fn(
#         torch.Tensor(my_norm_pred).float(),
#         torch.LongTensor([2, 1, 1, 1, 1, 2, 2, 1, 2, 1])))

# X_te_enc.shape

# Submit

In [ ]:
sub_df = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/sample_submission.csv')
print(uniques_tr2)

In [ ]:
out_df = pd.DataFrame({"discourse_id": test_df_selection['discourse_id']})
out_df[['Adequate', 'Effective', 'Ineffective']] = pred_j_out + np.abs(pred_j_out.min(axis=1, keepdims=True))*1.25
sub_df_f = pd.DataFrame(sub_df['discourse_id']).merge(out_df[['discourse_id', 'Ineffective', 'Adequate', 'Effective']],
                             on='discourse_id', how='left')
print(sub_df_f.isna().sum().sum())

In [ ]:
sub_df_f.to_csv('submission.csv', index=False)

In [ ]:
sub_df_f

In [ ]:
sub_df_f[['Ineffective','Adequate', 'Effective']].values/sub_df_f[['Adequate', 'Effective', 'Ineffective']].values.sum(axis=1, keepdims=True)